In [162]:
import numpy as np
import random
from pprint import pprint

In [163]:
class Cardan:
    def __init__(self, n: int = 4):
        self.n = n
        self.grid = None
        self.mask = None
        self.cipher = None

    def create_grid(self, shuffle: bool = False):
        quarter_size = self.n // 2
        quarter = np.arange(1, quarter_size**2+1).reshape((quarter_size, quarter_size))
        if shuffle:
            np.random.shuffle(quarter)
        up_left_quarter = quarter
        up_right_quarter = np.rot90(up_left_quarter, k=-1)
        down_right_quarter = np.rot90(up_right_quarter, k=-1)
        down_left_quarter = np.rot90(down_right_quarter, k=-1)

        up_half = np.concatenate((up_left_quarter, up_right_quarter), axis=1)
        down_half = np.concatenate((down_left_quarter, down_right_quarter), axis=1)

        self.grid = np.concatenate((up_half, down_half), axis=0)
        return self.grid

    def create_mask(self):
        stat = {}
        for i in range(self.n):
            for j in range(self.n):
                stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]

        self.mask = np.zeros((self.n, self.n))
        for fields in stat.values():
            i, j = random.choice(fields)
            self.mask[i,j] = 1
        return self.mask
    
    def create_cipher(self, text: str):
        text += "_" * (self.n**2 - len(text))
        text_i = 0
        mask = self.mask.copy()
        self.cipher = [["_" for _ in range(self.n)] for _ in range(self.n)]
        for _ in range(4):
            pprint(self.cipher)
            print("="*100)
            pprint(mask)
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        self.cipher[i][j] = text[text_i]
                        text_i += 1
            mask = np.rot90(mask, k=-1)
        return self.cipher
    
    def get_cryptogram(self):
        return "".join(["".join(row) for row in self.cipher])
    
    def get_cryptogram_key(self):
        return "".join(["".join([str(int(i)) for i in row]) for row in self.mask])
    
    def set_cryptogram(self, cryptogram: str) -> list[list[str]]:
        self.n = int(len(cryptogram)**0.5)
        self.cipher = [
            list(cryptogram[self.n*i:self.n*i+self.n]) for i in range(self.n)
        ]
        return self.cipher
    
    def set_cryptogram_key(self, cryptogram_key: str) -> np.ndarray:
        self.n = int(len(cryptogram_key)**0.5)
        self.mask = np.array([
            [int(sym) for sym in cryptogram_key[self.n*i:self.n*i+self.n]] for i in range(self.n)
        ])

        return self.mask
        
    def decode_cipher(self) -> str:
        result = ""
        mask = self.mask.copy()
        for _ in range(4):
            pprint(self.cipher)
            print(result[len(result)-self.n:len(result)])
            print(result)
            print("="*100)
            pprint(mask)
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        result += self.cipher[i][j]
            mask = np.rot90(mask, k=-1)
        return result

        

In [164]:
cardan = Cardan(n=4)

In [165]:
cardan.create_grid()

array([[1, 2, 3, 1],
       [3, 4, 4, 2],
       [2, 4, 4, 3],
       [1, 3, 2, 1]])

In [166]:
cardan.create_mask()

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [167]:
cardan.create_cipher("hello, guys, bli")

[['_', '_', '_', '_'],
 ['_', '_', '_', '_'],
 ['_', '_', '_', '_'],
 ['_', '_', '_', '_']]
array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.]])
[['h', '_', '_', '_'],
 ['_', '_', '_', '_'],
 ['e', 'l', '_', '_'],
 ['_', 'l', '_', '_']]
array([[0., 1., 0., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])
[['h', 'o', '_', ','],
 [' ', 'g', '_', '_'],
 ['e', 'l', '_', '_'],
 ['_', 'l', '_', '_']]
array([[0., 0., 1., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])
[['h', 'o', 'u', ','],
 [' ', 'g', 'y', 's'],
 ['e', 'l', '_', '_'],
 ['_', 'l', '_', ',']]
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [1., 0., 1., 0.]])


[['h', 'o', 'u', ','],
 [' ', 'g', 'y', 's'],
 ['e', 'l', ' ', 'b'],
 ['l', 'l', 'i', ',']]

In [168]:
cardan.get_cryptogram()

'hou, gysel blli,'

In [169]:
cardan.get_cryptogram_key()

'1000000011000100'

In [170]:
cardan.set_cryptogram("c1d29506e3f4a7b8")

[['c', '1', 'd', '2'],
 ['9', '5', '0', '6'],
 ['e', '3', 'f', '4'],
 ['a', '7', 'b', '8']]

In [171]:
cardan.set_cryptogram_key("0101000001010000")

array([[0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0]])

In [172]:
cardan.decode_cipher()

[['c', '1', 'd', '2'],
 ['9', '5', '0', '6'],
 ['e', '3', 'f', '4'],
 ['a', '7', 'b', '8']]


array([[0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0]])
[['c', '1', 'd', '2'],
 ['9', '5', '0', '6'],
 ['e', '3', 'f', '4'],
 ['a', '7', 'b', '8']]
1234
1234
array([[0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 1]])
[['c', '1', 'd', '2'],
 ['9', '5', '0', '6'],
 ['e', '3', 'f', '4'],
 ['a', '7', 'b', '8']]
5678
12345678
array([[0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 1, 0]])
[['c', '1', 'd', '2'],
 ['9', '5', '0', '6'],
 ['e', '3', 'f', '4'],
 ['a', '7', 'b', '8']]
90ab
1234567890ab
array([[1, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 0]])


'1234567890abcdef'